In [1]:
import requests
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

### API Requests (1. for auth; 2. for hotel listings and searching)

In [ ]:
# Need to run in order to retrieve an access token for the current user
# Place the output token in .env.
# If you already have a key, DO NOT RUN THIS PORTION

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
client_id = os.getenv("AMADEUS_API_KEY")
client_secret = os.getenv("AMADEUS_SECRET")
print(client_id)
print(client_secret)

# Prepare the data for the token request
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

header = {
    "Content-Type": "application/x-www-form-urlencoded"
}
# Make the token request
response = requests.post(token_url, data=data, headers=header)

# Check if the request was successful
if response.status_code == 200:
    token_info = response.json()
    access_token = token_info['access_token']
    print("Access Token:", access_token)
else:
    print("Failed to obtain access token:", response.status_code)
    print(response.text)


In [3]:
access_token = os.getenv("AMADEUS_ACCESS_TOKEN")
print(access_token)

DmzqoDsoJlnQtQZDvVC9TiGvH9rz


In [ ]:
# must refer to cities using IATA (airport) city codes

api = "https://test.api.amadeus.com"
hotel_list = "/v1/reference-data/locations/hotels/by-city"
hotel_offers = "/v3/shopping/hotel-offers"
auth_header = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": "Bearer " + access_token
}

params = {
    "cityCode": "LAX"
}
listings_in_city = requests.get(api+hotel_list, headers=auth_header, params=params).json()["data"]
print(listings_in_city)


[{'chainCode': 'BW', 'iataCode': 'LAX', 'dupeId': 700062770, 'name': 'BEST WESTERN PLUS EAGLE ROCK', 'hotelId': 'BWLAX518', 'geoCode': {'latitude': 34.14289, 'longitude': -118.22881}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2024-04-30T16:44:11'}, {'chainCode': 'WV', 'iataCode': 'LAX', 'dupeId': 700005329, 'name': 'HOTEL ERWIN VENICE BEACH', 'hotelId': 'WVLAX602', 'geoCode': {'latitude': 33.98695, 'longitude': -118.47245}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2023-06-15T10:29:35'}, {'chainCode': 'RC', 'iataCode': 'LAX', 'dupeId': 700115807, 'name': 'RESIDENCE INN MIRADA MARRIOTT', 'hotelId': 'RCLAXLMR', 'geoCode': {'latitude': 33.88299, 'longitude': -118.02399}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2024-04-22T06:07:51'}, {'chainCode': 'LZ', 'iataCode': 'LAX', 'dupeId': 700009539, 'name': 'LOEWS SANTA MONICA BEACH HOTEL', 'hotelId': 'LZLAXLSL', 'geoCode': {'latitude': 34.00988, 'longitude': -118.49309}, 'address': {'countryCode': 'US'}, 'lastUpdate': '202

In [21]:
hotel_search_params = [hotel["hotelId"] for hotel in listings_in_city]
# print(hotel_search_params)
responses = []
inc = 50
for i in range(0,len(hotel_search_params), inc):
    bound = min(i+inc, len(hotel_search_params))
    search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[i:bound]}).json()
    # hotel_search_params
    print(search)    
    if ("data" in search):
        responses += search["data"]

# search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[0:50]}).json()
# print(search)
print(responses)


{'data': [{'type': 'hotel-offers', 'hotel': {'type': 'hotel', 'hotelId': 'BWLAX204', 'chainCode': 'BW', 'dupeId': '700029042', 'name': 'Best Western Plus Hollywood Hills Hotel', 'cityCode': 'LAX', 'latitude': 34.10533, 'longitude': -118.32346}, 'available': True, 'offers': [{'id': 'CIOVZW5C5O', 'checkInDate': '2025-04-15', 'checkOutDate': '2025-04-16', 'rateCode': 'GOV', 'rateFamilyEstimated': {'code': 'GOV', 'type': 'C'}, 'category': 'GOV', 'commission': {'percentage': '10'}, 'room': {'type': 'B2Q', 'typeEstimated': {'category': 'STANDARD_ROOM', 'beds': 2, 'bedType': 'QUEEN'}, 'description': {'text': 'GOVT MILITARY RATE \n2 QUEEN BEDS,NSMK,42IN LCD TV,MICRFG,\nHI SPEED NET', 'lang': 'EN'}}, 'guests': {'adults': 1}, 'price': {'currency': 'USD', 'base': '129.20', 'total': '147.68', 'taxes': [{'code': 'TOTAL_TAX', 'amount': '18.48', 'currency': 'USD', 'included': False}], 'variations': {'average': {'base': '129.20'}, 'changes': [{'startDate': '2025-04-15', 'endDate': '2025-04-16', 'base'

In [42]:
print(responses[0])
print([{"prices": [p["price"]["total"] for p in r["offers"]], "name": r["hotel"]["name"]} for r in responses])

{'type': 'hotel-offers', 'hotel': {'type': 'hotel', 'hotelId': 'BWLAX204', 'chainCode': 'BW', 'dupeId': '700029042', 'name': 'Best Western Plus Hollywood Hills Hotel', 'cityCode': 'LAX', 'latitude': 34.10533, 'longitude': -118.32346}, 'available': True, 'offers': [{'id': 'CIOVZW5C5O', 'checkInDate': '2025-04-15', 'checkOutDate': '2025-04-16', 'rateCode': 'GOV', 'rateFamilyEstimated': {'code': 'GOV', 'type': 'C'}, 'category': 'GOV', 'commission': {'percentage': '10'}, 'room': {'type': 'B2Q', 'typeEstimated': {'category': 'STANDARD_ROOM', 'beds': 2, 'bedType': 'QUEEN'}, 'description': {'text': 'GOVT MILITARY RATE \n2 QUEEN BEDS,NSMK,42IN LCD TV,MICRFG,\nHI SPEED NET', 'lang': 'EN'}}, 'guests': {'adults': 1}, 'price': {'currency': 'USD', 'base': '129.20', 'total': '147.68', 'taxes': [{'code': 'TOTAL_TAX', 'amount': '18.48', 'currency': 'USD', 'included': False}], 'variations': {'average': {'base': '129.20'}, 'changes': [{'startDate': '2025-04-15', 'endDate': '2025-04-16', 'base': '129.20'

In [ ]:
# requires city_code in IATA format
# can be expanded to use filtering options in the search params
def get_hotels(city_code):
    # fetch access token
    token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    client_id = os.getenv("AMADEUS_API_KEY")
    client_secret = os.getenv("AMADEUS_SECRET")
    print(client_id)
    print(client_secret)

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    header = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    # Make the token request
    response = requests.post(token_url, data=data, headers=header)
    if response.status_code != 200:
        print("Failed to obtain access token:", response.status_code)
        print(response.text)
        return
    token_info = response.json()
    access_token = token_info['access_token']
    print("Access Token:", access_token)

    #make hotel list request (collect all hotels in city)
    api = "https://test.api.amadeus.com"
    hotel_list = "/v1/reference-data/locations/hotels/by-city"
    hotel_offers = "/v3/shopping/hotel-offers"
    auth_header = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": "Bearer " + access_token
    }

    params = {
        "cityCode": city_code
    }

    listings_in_city = requests.get(api+hotel_list, headers=auth_header, params=params).json()
    print(listings_in_city)
    
    # hotel search for next requests
    
    hotel_search_params = [hotel["hotelId"] for hotel in listings_in_city["data"]]
    # print(hotel_search_params)
    responses = []
    inc = 50
    for i in range(0,len(hotel_search_params), inc):
        bound = min(i+inc, len(hotel_search_params))
        search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[i:bound]}).json()
        # hotel_search_params
        print(search)    
        if ("data" in search):
            responses += search["data"]

    # search = requests.get(api + hotel_offers, headers=auth_header, params = {"hotelIds": hotel_search_params[0:50]}).json()
    # print(search)
    print(responses)
    out = [{"prices": [p["price"]["total"] for p in r["offers"]], "name": r["hotel"]["name"]} for r in responses]
    return out

get_hotels("LAX")

DmLKsaZ4jk9cUoFPgBZvQVdXTRABlPdh
zyGS6AQzNMt0oj50
Access Token: BBhJwFUPnxdMUO0zadzqiGRdMbYa
{'data': [{'chainCode': 'YX', 'iataCode': 'LAX', 'dupeId': 700145303, 'name': 'AMERICAS BEST VALUE INN', 'hotelId': 'YXLAXAVB', 'geoCode': {'latitude': 34.05134, 'longitude': -118.2645}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2023-06-15T10:21:12'}, {'chainCode': 'WV', 'iataCode': 'LAX', 'dupeId': 700113518, 'name': 'THE CAPRI', 'hotelId': 'WVLAX192', 'geoCode': {'latitude': 34.44905, 'longitude': -119.23262}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2023-06-15T09:51:53'}, {'chainCode': 'HX', 'iataCode': 'LAX', 'dupeId': 700216713, 'name': 'HAMPTON INN LAX HAWTHORNE', 'hotelId': 'HXLAX903', 'geoCode': {'latitude': 33.93063, 'longitude': -118.35105}, 'address': {'countryCode': 'US'}, 'lastUpdate': '2023-06-15T11:07:10'}, {'chainCode': 'RC', 'iataCode': 'LAX', 'dupeId': 700151572, 'name': 'RESIDENCE INN EL SEGU MARRIOTT', 'hotelId': 'RCLAXELR', 'geoCode': {'latitude': 33.91653, '

[{'prices': ['214.46'], 'name': 'Renaissance Los Angeles Airport Hotel'},
 {'prices': ['263.28'],
  'name': 'Courtyard by Marriott Los Angeles Baldwin Park'},
 {'prices': ['223.01'],
  'name': 'Courtyard by Marriott Los Angeles Pasadena Monrovia'},
 {'prices': ['219.61'], 'name': 'DOUBLETREE BY HILTON LOS ANGELES'},
 {'prices': ['186.23'], 'name': 'DOUBLETREE BY HILTON LA NORWALK'},
 {'prices': ['189.61'], 'name': 'DBLTREE LAX EL SEGUNDO'},
 {'prices': ['241.84'], 'name': 'Fairfield Anaheim Resort'},
 {'prices': ['178.18'],
  'name': 'Fairfield Inn by Marriott Santa Clarita Valencia'},
 {'prices': ['202.62'], 'name': 'HLTN GRDN INN LAX REDONDO BCH'},
 {'prices': ['225.08'], 'name': 'HOLIDAY INN EXPRESS VAN NUYS'},
 {'prices': ['236.64'], 'name': 'HOLIDAY INN EXP DOWNTOWN WEST'},
 {'prices': ['212.06'], 'name': 'HAMPTON INN LAX HAWTHORNE'},
 {'prices': ['224.18'], 'name': 'Los Angeles Airport Marriott'},
 {'prices': ['722.45'], 'name': 'VEA Newport Beach a Marriott Resort and Spa'},
 {'